<center><b> 1D time-dependent Frantz Nodvik equations </b></center>

<br> du1/dt + gamma*K*u1*u2 = 0           u1 = Delta = # density of excited states 
<br> du2/dt + c*du2/dx - K*u1*u2 = 0      u2 = n_ph  = # density of photons
<br>                                      c = speed of light
<br>                                      K = sigma * c
<br> IC:  TBD...
<br> BC:  TBD...
<br> This code is distributed under the Apache License, Version 2.0, January 2004.
<br> Copyright (c) 2020 RadiaSoft LLC, All rights reserved
<br> Author:  David Bruhwiler                                     
<br> The laser pulse duration $T_{laser}$ is typically 200 ps to 1 ns.
<br> The Ti-Sapphire phase relaxation time $T_{crystal}$ is between $10 fs$ and $10 ps$.
<br> Because $T_{crystal} << T_{laser}$, we can use the Frantz-Nodvik (F-N) rate equations.
<br> This approach can be used for other crystals as well.

In [2]:
from dolfin import *

In [5]:
def frantz_nodvik_1d_solver(sim_time, n_dx, c_gamma, c_sigma, c_width, c_es_dens, t_pulse, p_density):
    """Frantz-Nodvik 1D solver for wavefront amplification in a pumped crystal amplifier

    Args:
        sim_time (float): simulation time [s]
        n_dx (int): resolution of the 1D mesh along the x-axis
        c_gamma (float): ratio of degeneracies for crystal states []
        c_sigma (float): photon emission crosssection of the crystal [m^2]
        c_width (float): width of the crystal amplifier [m]
        c_density (float): # density of excited states in the crystal [cm^-3]
        lp_density (float): # density of photons in the laser pulse [cm^-3]
        lp_width (float): full width of the laser pulse [s]

    Returns:
        NA
    """
    import scipy.constants
    c_light = scipy.constants.c  # speed of light [m/s]
    K_sc = c_sigma * c_light     # [m^3/s]

    # to be deleted...
    print ('')
    print ('Input parameters --')
    print ('  simulation time [s]      = %g' % (sim_time) )
    print ('  # of mesh cells          = %d' % (n_dx) )
    print ('  c_gamma [ ]              = %g' % (c_gamma) )
    print ('  c_sigma [m^2]            = %g' % (c_sigma) )
    print ('  crystal width [m]        = %g' % (c_width) )
    print ('  # dens ex. states [m^-3] = %g' % (c_density) )
    print ('  # dens photons [m^-3]    = %g' % (lp_density) )
    print ('  width of laser pulse [s] = %g' % (lp_width) )

    # create a 1D mesh on the interval [0.,c_width].
    x_left = 0.
    x_right = c_width
    mesh = IntervalMesh(n_dx, x_left, x_right)

    # let's try flat line segments, with no requirement of continuity
    p1 = FiniteElement("Discontinuous Lagrange","interval",0)
    
    # define an 'element' appropriate for two 1D functions
    element = MixedElement([p1, p1])
    
    # define the test functions: v=[v_1, v_2]
    v = FunctionSpace(mesh, element)
    v_1, v_2 = TestFunctions(v)

    # define solution functions: u=[u_1, u_2]
    u = Function(v)
    u_1, u_2 = split(u)

    # solutions from the previous time step: u_n=[u_n1, u_n2]
    u_n = Function(v)
    u_n1, u_n2 = split(u_n)

    # boundary conditions imposed at x=0.
    # assume the incident wavefront has a square profile
    u_left = Expression('t>=0. && t<=lp_width ? lp_density : 0.', degree=1, t=0.)
    def on_left(x, on_boundary):
        return (on_boundary and near(x[0], x_left))
    
    # apply the BCs
    u_left.t = sim_time
    bc = DirichletBC(V, u_left, on_left)

    # define the variational problem via function F
    F = ( dot ( u - u_old, v ) / DT \
    + nu * inner ( grad ( u ), grad ( v ) ) \
    + inner ( u * u.dx(0), v ) \
    - dot ( f, v ) \
    ) * dx

    # specify the Jacobian
    J = derivative(F, u)

    # solve the system of equations
    solve(F==0, u, bc, J=J)

    return

In [ ]:
    
    # apply the initial conditions, if necessary
    if sim_time == 0.:
        u0_1 = Constant(c_density)  # assuming density of states is constant
        u_1 = project(u0_1, v)
        u0_2 = Constant(0.)         # assuming no photons in crystal at t=0
        u_2 = project(u0_2, v)

        # set U and U0 by interpolation.
        u_n.assign(u)
